In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [23]:
USE_CUDA = torch.cuda.is_available()
#  torch.cuda.is_available() 함수는 현재 컴퓨터에서 CUDA를 이용할 수 있는 지 알아보는 함수
# 오류가 있다면 false 반환
# 제대로 설치시 true (CUDA용 파이토치, CUDA도 제대로 설치)
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
# torch.device에 CUDA를 지원하면 "cuda"를 
# 아니면 "cpu"를 설정한 후 변수에 저장 
# 나중에 텐서와 가중치에 대한 연산을 CPU와 GPU 중 에서 실행할지 결정할 떄 사용 

In [24]:
EPOCHS = 30
# 전체 반복 횟수

BATCH_SIZE = 64
# FashionMNIST 부터는 데이터가 많아져 한 번에 사용하는 것은 효율적이지 않기 때문에
# 여러 개의 미니배치로 잘라서 사용
# 각 미니배치의 크기는 64개 

transform = transforms.Compose([
 transforms.ToTensor()
])
# ToTensor로 이미지를 파이토치 텐서로 변환하고
# Compose로 만들어 둔 이미지 변환 설정을 적용하는 데 쓰인다. 


## FashionMNIST 데이터셋을 가져온다.
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True,
    transform = transform
)

testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform 
)
# download = True로 설정해서 
# 자동으로 root 로 지정한 폴더 ( './.data/', ) 에 
# 데이터셋이 없다면 자동으로 다운로드 


## DataLoader 사용 
train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

In [25]:
## 이미지 분류를 위한 인공신경망 구현 입력 X와 레이블 Y를 입력 받아 
## 학습한 다음 새로운 X가 왔을 때 어떤 패션 아이템인지 예측한다. 

class Net(nn.Module):
    def __init__(self): # 선형결합을 수행하는 객체를 만든다. 
        super(Net,self).__init__()
        self.fc1 = nn.Linear(784, 256) #픽셀값 784개를 입력받아 256 출력 
        self.fc2 = nn.Linear(256, 128)#픽셀값 256개를 입력받아 128 출력 
        self.fc3 = nn.Linear(128,10) #픽셀값 128개를 입력받아 10 출력
        
    def forward(self,x):
        x = x.view(-1,784) #view( ) 로 1차원 행렬을 만든다. 
        x = F.relu(self.fc1(x)) # fc1()를 거쳐 relu( )활성화 함수 
        x = F.relu(self.fc2(x)) # fc2()를 거쳐 relu( )활성화 함수
        x = self.fc3(x) # fc3()를 거침 
        return x

In [26]:
model = Net().to(DEVICE) 
# to 함수로 연산을 어디서 수행할지 결정 
# CPU 사용시 to gkatn t0 함수 사용할 필요 x
# to("cuda") ->  GPU 사용 
# DEVICE 변수를 설정했기 때문에 
#자동으로 CUDA를 사용할 경우 GPU로  아닐경우 CPU사용 
optimizer = optim.SGD(model.parameters(), lr =0.01) 
# 최적화 알고리즘으로 파이토치 내장 모듈인 optim.SGD 사용 
# 모델 내부의 정보를 넘겨주는 model.parameters() 함수와 
# 임의로 설정한 학습률(lr) = 0.01 입력 

In [27]:
## 학습에 들어가는 모든 연산 

# model- 학습에 필요한 모든 요소 
# train_loader - 데이터 공급
# optimizer - 최적화 담당 


# 모델에 입력하는 data = [배치크기, 색, 높이, 넓이] = [64,1,28,28 ]
 
def train(model, train_loader, optimizer):
    model.train() #학습모드로 바꾼다.
    
    
    for batch_idx,(data,target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        # 앞서 모델의 가중치를 GPU로 보냈다면 
        # 학습 데이터도 같은 장치로 보내야 연산을 수행
        optimizer.zero_grad()
        # 반복 시마다 기울기를 계산 
        output = model(data)
        # 학습 데이터에 대한 모델의 예측값을 받아온다. 
        loss = F.cross_entropy(output,target)
        # 앞 장에서는 클래스가 2개 라 이진 교차 엔트로피를 사용하였지만
        #이번에는 10개가 존재하기 때문에 교차 엔트로피 사용 
        # 앞 장 처럼 criterion() 함수를 만들 수 있으나
        # 가중치를 보관할 필요가 없으므로 
        # torch.nn.functional 의 cross_entropy() 함수를 직접 사요 ㅇ
        loss.backward()
        # 기울기 가 계싼 
        optimizer.step()
        # 계싼한 기울기를 앞 서 정의한 알고리즈메 맞춰 가중치를 수정 

In [31]:
def evaluate(model, test_loader):
    model.eval()
    # 평가 모드로 바꾼다. 
    test_loss = 0 
    # 테스트 오차 수 
    correct = 0 
    # 테스트 예측이 맞은 수 
    
    with torch.no_grad():# 평가 과정에서는 기울기를 계산하지 않아도 도니다. 
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            # 학습 때와 같이 데이터를 DEVICE로 내보내고 
            # 모델의 예측값(output)를 받아옴
            
            test_loss += F.cross_entropy(output,target, reduction='sum').item()
            # 모든 테스트 오차 더하기
            
            pred = output.max(1, keepdim = True )[1]
            # 가장 큰 값을 가진 클래스가 모델의 예측
            correct += pred.eq(target.view_as(pred)).sum().item()
            # 예측과 정답을 비교하여 일치할 경우 correct에 1을 더한다.
            # view_as()는 target 텐서를 pred의 모양대로 정렬 
            # eq()는 값이 일치하면 1  불일치하면 0
            # eq()를 지나고 나면 배치 크기의 배열이 0이나 1로 채워짐
            # sum() 함수로 배열 내의 모든 값을 더하면
            # 모델이 정답을 맞춘 수를 구할 수 있다. 
            
    test_loss /= len(test_loader.dataset) # 오차 수 / 전체 테스트셋 수
    test_accuracy = 100. *correct / len (test_loader.dataset) 
    #정확도= 정답을 맞춘 수 / 전체 테스트 셋 수  *100
    return test_loss, test_accuracy

In [32]:
for epoch in range(1, EPOCHS + 1):
    # 이폭마다 학습과 테스트셋을 이용하 ㄴ검증을 반복하고 결과를 출력 
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch,test_loss,test_accuracy))

[1] Test Loss: 0.5873, Accuracy: 79.08%
[2] Test Loss: 0.5858, Accuracy: 78.79%
[3] Test Loss: 0.5149, Accuracy: 81.51%
[4] Test Loss: 0.5078, Accuracy: 81.84%
[5] Test Loss: 0.5264, Accuracy: 80.22%
[6] Test Loss: 0.4790, Accuracy: 82.85%
[7] Test Loss: 0.4885, Accuracy: 82.89%
[8] Test Loss: 0.4548, Accuracy: 83.95%
[9] Test Loss: 0.4632, Accuracy: 83.26%
[10] Test Loss: 0.4418, Accuracy: 84.40%
[11] Test Loss: 0.4559, Accuracy: 83.85%
[12] Test Loss: 0.4449, Accuracy: 84.08%
[13] Test Loss: 0.4301, Accuracy: 84.94%
[14] Test Loss: 0.4328, Accuracy: 85.04%
[15] Test Loss: 0.4106, Accuracy: 85.47%
[16] Test Loss: 0.4327, Accuracy: 84.59%
[17] Test Loss: 0.4065, Accuracy: 85.59%
[18] Test Loss: 0.4009, Accuracy: 85.69%
[19] Test Loss: 0.4178, Accuracy: 84.94%
[20] Test Loss: 0.3890, Accuracy: 86.34%
[21] Test Loss: 0.3969, Accuracy: 85.81%
[22] Test Loss: 0.3949, Accuracy: 85.81%
[23] Test Loss: 0.3821, Accuracy: 86.52%
[24] Test Loss: 0.4028, Accuracy: 85.50%
[25] Test Loss: 0.3811, A